In [9]:
from google.colab import files
uploaded = files.upload()

Saving ocr-based-question-extraction-8c660bbdac8e.json to ocr-based-question-extraction-8c660bbdac8e.json


In [ ]:
!pip install google-cloud-vision
!pip install pdf2image PyMuPDF Pillow

!apt-get update -qq
!apt-get install -y poppler-utils  # Installs pdftoppm, pdfinfo, etc.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.1/529.1 kB 9.3 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 66.7 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Fetched 186 kB in 0s (1,189 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up p

In [10]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = list(uploaded.keys())[0]

In [11]:
from pdf2image import convert_from_path

uploaded = files.upload()
pdf_file = list(uploaded.keys())[0]
images = convert_from_path(pdf_file)

# Save images temporarily
for i, image in enumerate(images):
    image.save(f"page_{i+1}.png")
    print(f"Page {i+1} saved as page_{i+1}.png")

Saving 16CS302 r 16CT303 Database Management Systems.pdf to 16CS302 r 16CT303 Database Management Systems (2).pdf
Page 1 saved as page_1.png
Page 2 saved as page_2.png


In [13]:
from google.cloud import vision
from PIL import Image, ImageDraw
import re
import os
import math

# Initialize the client
client = vision.ImageAnnotatorClient()

def merge_close_characters(image_path, max_distance=15):
    """
    Improved OCR function that merges nearby characters on the same line
    """
    # Read image
    with open(image_path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations

    if len(texts) <= 1:
        return []

    # Process text blocks
    text_blocks = []
    for text in texts[1:]:  # Skip full text annotation
        vertices = [(vertex.x, vertex.y) for vertex in text.bounding_poly.vertices]
        center_x = sum(v[0] for v in vertices) / 4
        center_y = sum(v[1] for v in vertices) / 4
        text_blocks.append({
            "text": text.description,
            "vertices": vertices,
            "center_x": center_x,
            "center_y": center_y
        })

    # Sort by Y then X (top-to-bottom, left-to-right)
    text_blocks.sort(key=lambda x: (x["center_y"], x["center_x"]))

    # Group into lines
    lines = []
    current_line = []
    y_threshold = 10  # Max vertical diff for same line

    for block in text_blocks:
        if not current_line:
            current_line.append(block)
            continue

        prev_y = current_line[-1]["center_y"]
        current_y = block["center_y"]

        if abs(current_y - prev_y) <= y_threshold:
            current_line.append(block)
        else:
            lines.append(current_line)
            current_line = [block]

    if current_line:
        lines.append(current_line)

    # Merge within each line
    merged_blocks = []

    for line in lines:
        line.sort(key=lambda x: x["center_x"])
        current_merged = None

        for block in line:
            if not current_merged:
                current_merged = block.copy()
                continue

            prev_right = max(v[0] for v in current_merged["vertices"])
            curr_left = min(v[0] for v in block["vertices"])
            gap = curr_left - prev_right

            if gap <= max_distance:
                current_merged["text"] += block["text"]
                combined_vertices = [
                    current_merged["vertices"][0],
                    (block["vertices"][1][0], current_merged["vertices"][1][1]),
                    (block["vertices"][2][0], block["vertices"][2][1]),
                    (current_merged["vertices"][3][0], block["vertices"][3][1])
                ]
                current_merged["vertices"] = combined_vertices
            else:
                merged_blocks.append(current_merged)
                current_merged = block.copy()

        if current_merged:
            merged_blocks.append(current_merged)

    return merged_blocks

def detect_full_questions(image_path):
    """
    Modified version using the improved OCR function
    """
    merged_blocks = merge_close_characters(image_path)

    # Regex to detect question patterns (updated to include parentheses)
    question_pattern = re.compile(r'^\d+\.[a-z]{1}\.$')

    questions = []
    current_question = None

    for block in merged_blocks:
        if question_pattern.match(block["text"]):
            if current_question:
                questions.append(current_question)
            current_question = {
                "text": block["text"],
                "vertices": block["vertices"]
            }
        elif current_question:
            # Extend the current question
            current_question["text"] += " " + block["text"]
            current_question["vertices"].extend(block["vertices"])

    if current_question:
        questions.append(current_question)

    return questions

# (Rest of the original code remains exactly the same)
def crop_and_save_questions(image_path, questions, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    image_pil = Image.open(image_path)

    for i, question in enumerate(questions):
        vertices = question["vertices"]
        x_coords = [vertex[0] for vertex in vertices]
        y_coords = [vertex[1] for vertex in vertices]
        x1, y1 = min(x_coords), min(y_coords)
        x2, y2 = max(x_coords), max(y_coords)
        padding = 24
        x1 = max(0, x1 - padding)
        y1 = max(0, y1 - padding)
        x2 = min(image_pil.width, x2 + padding)
        y2_original = min(image_pil.height, y2 + padding)

        if i < len(questions) - 1:
            next_vertices = questions[i + 1]["vertices"]
            next_y1 = min([vertex[1] for vertex in next_vertices])
            buffer = 5
            y2 = min(y2_original, next_y1 - buffer)
        else:
            y2 = y2_original

        y2 = max(y1, y2)
        question_image = image_pil.crop((x1, y1, x2, y2))
        output_path = f"{output_folder}/{os.path.basename(image_path)}_question_{i+1}.png"
        question_image.save(output_path)
        print(f"Saved: {output_path}")

# Process each page (main workflow remains unchanged)
output_folder = "question_images_test_4"
for i in range(len(images)):
    page_image_path = f"page_{i+1}.png"
    print(f"Processing {page_image_path}...")
    questions = detect_full_questions(page_image_path)
    crop_and_save_questions(page_image_path, questions, output_folder)

Processing page_1.png...
Saved: question_images_test_4/page_1.png_question_1.png
Saved: question_images_test_4/page_1.png_question_2.png
Saved: question_images_test_4/page_1.png_question_3.png
Saved: question_images_test_4/page_1.png_question_4.png
Processing page_2.png...
Saved: question_images_test_4/page_2.png_question_1.png
Saved: question_images_test_4/page_2.png_question_2.png
Saved: question_images_test_4/page_2.png_question_3.png
Saved: question_images_test_4/page_2.png_question_4.png
Saved: question_images_test_4/page_2.png_question_5.png
Saved: question_images_test_4/page_2.png_question_6.png
Saved: question_images_test_4/page_2.png_question_7.png
Saved: question_images_test_4/page_2.png_question_8.png
Saved: question_images_test_4/page_2.png_question_9.png
Saved: question_images_test_4/page_2.png_question_10.png
Saved: question_images_test_4/page_2.png_question_11.png
